In [ ]:
def detect_faces_in_rotations(frame, face_cascade, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)):
    # Define the rotation angles
    rotation_angles = [0, 90, 270]
    detected_faces = []

    original_w, original_h = frame.shape[1], frame.shape[0]

    for angle in rotation_angles:
        # Rotate the image
        rotated_frame, M = rotate_image(frame, angle)
        
#         # Convert the rotated image to grayscale
#         gray_frame = cv2.cvtColor(rotated_frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(rotated_frame)
        
        if len(faces) > 0:
            # If faces are detected, adjust their positions based on the rotation angle
            for (x, y, w, h) in faces:
                # Calculate the inverse rotation for the detected face coordinates
                inv_M = cv2.invertAffineTransform(M)
                original_coords = np.array([[x + w/2, y + h/2]], dtype=np.float32)
                original_coords = np.array([original_coords])
                # Transform back the coordinates of the center of the face
                transformed_coords = cv2.transform(original_coords, inv_M)
                ox, oy = transformed_coords[0][0]
                ox, oy = int(ox - w/2), int(oy - h/2)
                
                # Store the corrected face box
                detected_faces.append((ox, oy, w, h))
            break  # Stop if faces are detected in the current orientation

    return detected_faces

In [ ]:
def detect_faces_in_rotations_2(frame, face_cascade, orientation_counter=None):
    # Define the rotation angles
    rotation_angles = [0]
    detected_faces = []

    original_w, original_h = frame.shape[1], frame.shape[0]
    most_faces_detected = 0
    best_angle = 0

    for angle in rotation_angles:
        # Rotate the image
        rotated_frame, M = rotate_image(frame, angle)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(rotated_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        if len(faces) > most_faces_detected:
            # If more faces are detected, update the counter and the best angle
            most_faces_detected = len(faces)
            best_angle = angle
            detected_faces = [(x, y, w, h) for (x, y, w, h) in faces]
            
    # Update orientation counter if it's provided
    if orientation_counter is not None:
        orientation_counter.update(best_angle, most_faces_detected)
        best_angle = orientation_counter.get_preferred_orientation()

    # Adjust detected faces based on the best angle
    adjusted_faces = []
    if most_faces_detected > 0:
        for (x, y, w, h) in detected_faces:
            # Calculate the inverse rotation for the detected face coordinates based on the best angle
            rotated_frame, M = rotate_image(frame, best_angle)
            inv_M = cv2.invertAffineTransform(M)
            original_coords = np.array([[x + w/2, y + h/2]], dtype=np.float32)
            original_coords = np.array([original_coords])
            # Transform back the coordinates of the center of the face
            transformed_coords = cv2.transform(original_coords, inv_M)
            ox, oy = transformed_coords[0][0]
            ox, oy = int(ox - w/2), int(oy - h/2)
            
            # Store the corrected face box
            adjusted_faces.append((ox, oy, w, h))
    
#     print(f'best_angle: {best_angle}, orientation_counter: {orientation_counter.counters}')

    return adjusted_faces

In [ ]:
def detect_faces_in_rotations_3(frame, face_cascade, orientation_counter=None):
    # Define the rotation angles
    rotation_angles = [0]
    detected_faces = dict()

    original_w, original_h = frame.shape[1], frame.shape[0]
    most_faces_detected = 0
    best_angle = 0

    for angle in rotation_angles:
        # Rotate the image
        rotated_frame, M = rotate_image(frame, angle)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(rotated_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        if len(faces) > most_faces_detected:
            # If more faces are detected, update the counter and the best angle
            most_faces_detected = len(faces)
            best_angle = angle
            detected_faces[angle] = [(x, y, w, h) for (x, y, w, h) in faces]
            
    # Update orientation counter if it's provided
    if orientation_counter is not None:
        orientation_counter.update(best_angle, most_faces_detected)
        best_angle = orientation_counter.get_preferred_orientation()

    # Adjust detected faces based on the best angle
    adjusted_faces = []
    if most_faces_detected > 0:
        for (x, y, w, h) in detected_faces[best_angle]:
            # Calculate the inverse rotation for the detected face coordinates based on the best angle
            rotated_frame, M = rotate_image(frame, best_angle)
            inv_M = cv2.invertAffineTransform(M)
            original_coords = np.array([[x + w/2, y + h/2]], dtype=np.float32)
            original_coords = np.array([original_coords])
            # Transform back the coordinates of the center of the face
            transformed_coords = cv2.transform(original_coords, inv_M)
            ox, oy = transformed_coords[0][0]
            ox, oy = int(ox - w/2), int(oy - h/2)
            
            # Store the corrected face box
            adjusted_faces.append((ox, oy, w, h))
    
#     print(f'best_angle: {best_angle}, orientation_counter: {orientation_counter.counters}')

    return adjusted_faces